## Konkordanser for NAOB mot Bokhylla

Versjon: **August 2020**
Inneholder korpus med to utvidelser, en fra 2019 og en fra 2020.

Velg `Run all` fra Cell-menyen over. 

Søkeord er forskjellig med stor og liten forbokstav. Forfatter og tittel kan trunkeres, for eksempel *Agnar Mykle* kan skrives som *mykl*. 

In [1]:
import pandas as pd
import requests
import inspect
import csv
import dhlab.module_update as mu
mu.update('dhlab_v2')
import dhlab_v2 as d2

Updated file `/mnt/disk1/Github/NAOB_konkordans/dhlab_v2.py`

In [2]:
def get_urns_from_text(document):
    """Find all URNs in a text-file"""
    
    import re

    with open(document) as fp:
        text = fp.read()
    #print(text)
    return re.findall("URN:NBN:no-nb_digibok_[0-9]{13}", text)

In [3]:
res = dict()

naob = get_urns_from_text("./naob_korpus.txt")
naobutvid = get_urns_from_text("./Litterære kilder tilleggsliste til UBB og NB 20190328.txt")
naobutvid2 = get_urns_from_text("./NAOBTilleggURN2020-06-10.txt")
NAOB = naob + naobutvid + naobutvid2


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf8 in position 121: invalid start byte

In [ ]:
naob_df = pd.DataFrame(naob, columns = ['urn'])
naob_df['corpus'] = 'naob_original'


naob_all_df = pd.DataFrame(NAOB, columns = ['urn'])
naob_df['corpus'] = 'naob_hele'


naob_utvidelse2019_df = pd.DataFrame(naobutvid, columns = ['urn'])
naob_utvidelse2019_df['corpus'] = 'naob_2019'


naob_utvidelse2020_df = pd.DataFrame(naobutvid2, columns = ['urn'])
naob_utvidelse2020_df['corpus'] = 'naob_2020'


In [ ]:
naob_corpus = pd.concat([naob_df, naob_all_df, naob_utvidelse2019_df, naob_utvidelse2020_df])

In [4]:
naob_corpus.to_csv('naob.csv')

NameError: name 'naob_corpus' is not defined

In [5]:
conc = d2.Concordance(naob_df, 'arbeid')

NameError: name 'naob_df' is not defined

In [6]:
conc.size

NameError: name 'conc' is not defined